In [1]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import load_boston
X,y=load_boston(return_X_y=True)#,as_frame=True)


In [33]:
# np.array_split(X,5)
arr_idx=np.arange(X.shape[0])
shuffled_idx=np.random.choice(arr_idx,size=len(arr_idx),replace=False)
# X[shuffled_idx,:].shape==X.shape
split_shuffled_idx=np.array_split(shuffled_idx,5)
# X[split_shuffled_idx[0],:]

(102, 13)

In [41]:
#Splitter Possibly mananged by the master node/leader_node

#Each function is a computation that will require Conputation cluster
def splitter(d1,d2):
  if len(d2.shape)==1:
    d2=d2.reshape(-1,1)
  assert(d1.shape[0]==d2.shape[0])
  arr_idx=np.arange(d1.shape[0])
  shuffled_idx=np.random.choice(arr_idx,size=len(arr_idx),replace=False)
  split_shuffled_idx=np.array_split(shuffled_idx,5)
  data_sets=[(d1[i,:],d2[i,:]) for i in split_shuffled_idx]
  return data_sets


In [106]:
#Mapper functions
def cov_xx_computer(a,b):
  return np.transpose(a)@a
def cov_xy_computer(a,b):
  return np.transpose(a)@b

def cov_computer(d):
  a=d[0]
  b=d[1]
  return (cov_xx_computer(a,b),cov_xy_computer(a,b))


# cov_computer(data_sets[0])

#Reducer Function
def sum_reducer(a,b):
  temp_xx=a[0]+b[0]
  temp_xy=a[1]+b[1]
  return [temp_xx,temp_xy]
  

def mapper_solver(dt):
  temp_xx=dt[0]
  temp_xy=dt[1]
  if (len(temp_xy.shape)==1):
    temp_xy=temp_xy.reshape(-1,1)
  final_coeff=np.linalg.solve(temp_xx,temp_xy)
  return final_coeff

In [112]:
from functools import reduce

data_sets=splitter(X,y)
lt_mapped_data=list(map(cov_computer,data_sets))

final_cov_data=[reduce(sum_reducer,lt_mapped_data)]
#We obtain sum of covariance matrices of dtat distributed along different clusters
#We need to pass this tuple object inside list so that the map function can work accordingly[(xx,xy)]


#then we have an aggregated cov_xx and cov_xy on which w eneed to apply/map one linalg function
tuple(map(mapper_solver,final_cov_data))



(array([[-9.28965170e-02],
        [ 4.87149552e-02],
        [-4.05997958e-03],
        [ 2.85399882e+00],
        [-2.86843637e+00],
        [ 5.92814778e+00],
        [-7.26933458e-03],
        [-9.68514157e-01],
        [ 1.71151128e-01],
        [-9.39621540e-03],
        [-3.92190926e-01],
        [ 1.49056102e-02],
        [-4.16304471e-01]]),)

# Rough WORK

In [21]:
y=y.reshape(-1,1)
xx=np.transpose(X)@X
xy=np.transpose(X)@y

np.linalg.solve(xx,xy)

array([[-9.28965170e-02],
       [ 4.87149552e-02],
       [-4.05997958e-03],
       [ 2.85399882e+00],
       [-2.86843637e+00],
       [ 5.92814778e+00],
       [-7.26933458e-03],
       [-9.68514157e-01],
       [ 1.71151128e-01],
       [-9.39621540e-03],
       [-3.92190926e-01],
       [ 1.49056102e-02],
       [-4.16304471e-01]])

In [115]:
list((1,2)),'\t',[(1,2)]

([1, 2], '\t', [(1, 2)])